# Let's generate the baseline

In [1]:
%run ../script/webnlg.py
import pandas as pd

pd.set_option('max_colwidth', 100)

test = WebNLGCorpus.load("test_with_lex")

In [57]:
def join_triples(g):
    
    return ' '.join(g.mtext.str.replace('[|_"]', ' ').str.replace(r'\s+', ' ').tolist())

baseline = test.mdf.groupby('idx').apply(join_triples).to_frame().reset_index()
baseline.rename(columns={0: 'text'}, inplace=True)
baseline['id'] = baseline.idx.str.split('_').apply(lambda v: v[1]).astype('int')

In [58]:
baseline.head()

,idx,text,id
0,0_0,"Abilene Regional Airport cityServed Abilene, Texas",0
1,0_1,"Adolfo Suárez Madrid–Barajas Airport location Madrid, Paracuellos de Jarama, San Sebastián de lo...",1
2,0_10,Amsterdam Airport Schiphol 5th runway SurfaceType Asphalt,10
3,0_100,A.D. Isidro Metapán league Salvadoran Primera División,100
4,0_1000,Alfons Gorbach deathPlace Austria,1000


In [59]:
import codecs

with codecs.open('models/baseline/output.txt', 'w', 'utf-8') as f:
    
    for _, text in baseline.sort_values('id').text.iteritems():
        
        f.write(text)
        f.write('\n')

In [1]:
!head models/baseline/output.txt

Abilene Regional Airport cityServed Abilene, Texas
Adolfo Suárez Madrid–Barajas Airport location Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas 
Adolfo Suárez Madrid–Barajas Airport runwayName 18L/36R 
Afonso Pena International Airport ICAO Location Identifier SBCT 
Afonso Pena International Airport cityServed Curitiba
Al-Taqaddum Air Base cityServed Fallujah
Al-Taqaddum Air Base runwayLength 3684.0
Alderney Airport runwayName 14/32 
Allama Iqbal International Airport runwayLength 3360.12
Amsterdam Airport Schiphol 1st runway Number 18


In [3]:
!python ../evaluation/webnlg2017/webnlg-automatic-evaluation-v2/evaluation_v2.py --team_name baseline --team_filepath models/baseline/output.txt --outdir models/baseline/

Files creating finished for:  baseline


In [4]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < models/baseline/baseline_all-cat.txt

BLEU = 23.56, 61.2/34.8/19.6/11.4 (BP=0.896, ratio=0.901, hyp_len=36820, ref_len=40883)
